# NIBRS 2018
> Prepared by [Dawn Graham](https://github.com/dawngraham).  
> First created during Code for Boston's National Day of Civic Hacking (NDoCH) 2019 as part of the [Clean Slate project](https://github.com/codeforboston/clean-slate).


## Purpose
- Summarize offenses in our NIBRS 2018 data.
- Determine the percent of records eligible for expungement involving people 21 and younger (assuming it is the person's only offense).

In [1]:
# Import packages
import pandas as pd
import pandas_profiling

---
## Data Sources & Limitations
- Note: Data has not yet been added to clean-slate repo due to concern about replicating data in multiple places.

### NIBRS 2018
- This is 2018 arrest data from NIBRS and the Boston Police Department.
- The combined data was provided by CfJJ because FBI data excludes certain jurisdictions.
- This only provides incident-level records. It cannot identify multiple incidents involving the same person, which would provide further information about eligibility for expungement.
- Descriptions from NIBRS User Manual: https://ucr.fbi.gov/nibrs/nibrs-user-manual

In [2]:
# Import data
nibrs2018 = pd.read_excel('../data/2018 NIBRS+BPD.xlsx')
nibrs2018.head()

,ORI,Incident Number,Incident Date,County,Agency Name,Arrestee Seq,Arrest Date,Arrest Month,BK_TIME,BRICArrestCode,BRIC Desc,Arrestee Offense,Type of Arrest,Arrestee Age,Arrestee Gender,Arrestee Race,Arrestee Ethnicity,Disposition Under 18
0,MA0010500,08-1373-OF,2008-11-08,Barnstable County,Dennis,1.0,2018-05-17,May,NaN,NaN,NaN,Motor Vehicle Theft,Summoned/Cited,44,Male,White,Not Hispanic or Latino,Missing
1,MA0010100,10-1359-OF,2010-06-19,Barnstable County,Barnstable,1.0,2018-06-22,June,NaN,NaN,NaN,Burglary/Breaking & Entering,Taken into Custody,24,Male,White,Not Hispanic or Latino,Missing
2,MA0110200,10-1400-OF,2010-12-11,Norfolk County,Bellingham,1.0,2018-05-21,May,NaN,NaN,NaN,All Other Offenses,Taken into Custody,46,Male,White,Not Hispanic or Latino,Missing
3,MA0112300,10-1069-OF,2010-09-21,Norfolk County,Stoughton,1.0,2018-11-11,November,NaN,NaN,NaN,False Pretenses/Swindle/Confidence Game,Summoned/Cited,12,Male,White,Not Hispanic or Latino,Referred to Other Authorities
4,MA0146000,201100008535,2011-01-27,Worcester County,Worcester,1.0,2018-01-28,January,NaN,NaN,NaN,Simple Assault,Taken into Custody,39,Female,White,Hispanic or Latino,Missing


In [3]:
# Convert dates to datetime
nibrs2018['Arrest Date'] = pd.to_datetime(nibrs2018['Arrest Date'])
nibrs2018['Incident Date'] = pd.to_datetime(nibrs2018['Incident Date'])

In [4]:
# Get incident age
nibrs2018['Incident Age'] = nibrs2018['Arrestee Age'] - (nibrs2018['Arrest Date'].dt.year - nibrs2018['Incident Date'].dt.year)

In [5]:
# Get only relevant columns
nibrs2018 = nibrs2018[['Incident Age', 'Arrestee Age', 'Arrestee Offense']]
nibrs2018.describe()

,Incident Age,Arrestee Age
count,100018.000000,111212.000000
mean,35.048711,35.010709
std,12.906360,12.858649
min,0.000000,0.000000
25%,25.000000,25.000000
50%,33.000000,33.000000
75%,44.000000,44.000000
max,99.000000,99.000000


#### Get incidents where incident or arrestee age is 21 or under

In [6]:
nibrs21 = nibrs2018[(nibrs2018['Incident Age'] <= 21) | (nibrs2018['Arrestee Age'] <= 21)]
nibrs21.describe()

,Incident Age,Arrestee Age
count,14109.000000,15676.000000
mean,18.225742,18.242919
std,2.448361,2.418865
min,0.000000,0.000000
25%,17.000000,17.000000
50%,19.000000,19.000000
75%,20.000000,20.000000
max,21.000000,24.000000


In [7]:
nibrs21.head()

,Incident Age,Arrestee Age,Arrestee Offense
1,16.0,24,Burglary/Breaking & Entering
3,4.0,12,False Pretenses/Swindle/Confidence Game
5,16.0,21,Aggravated Assault
6,18.0,23,Theft From Building
10,18.0,22,Burglary/Breaking & Entering


#### By Age

In [8]:
nibrs21['Incident Age'].value_counts().sort_index(ascending=False)

21.0    2753
20.0    2401
19.0    2272
18.0    2098
17.0    1403
16.0    1155
15.0     880
14.0     572
13.0     337
12.0     161
11.0      32
10.0      10
9.0        2
8.0        4
7.0        1
6.0        2
5.0        1
4.0        1
0.0       24
Name: Incident Age, dtype: int64

In [9]:
nibrs21['Arrestee Age'].value_counts().sort_index(ascending=False)

24       2
23       1
22      17
21    3043
20    2697
19    2485
18    2312
17    1599
16    1315
15     985
14     619
13     365
12     161
11      32
10       9
9        2
8        4
7        1
6        2
5        1
0       24
Name: Arrestee Age, dtype: int64

#### By Offense

In [10]:
nibrs21['Arrestee Offense'].value_counts().head(15)

All Other Offenses                          3410
Simple Assault                              2581
Aggravated Assault                           965
Drug/Narcotic Violations                     782
Liquor Law Violations                        727
Disorderly Conduct                           663
Destruction/Damage/Vandalism of Property     621
Shoplifting                                  619
Driving Under the Influence                  488
Drunkenness                                  446
All Other Larceny                            354
Intimidation                                 333
Burglary/Breaking & Entering                 294
Trespass of Real Property                    269
Robbery                                      234
Name: Arrestee Offense, dtype: int64

### Master Crime List Mapping
- This data maps offenses above in NIBRS 2018 to the *Master Crime List offense with Expunge categories.xlsx* spreadsheet provided by CfJJ.
- This had to be determined by manual review because different language is used in arrest records, etc.
- Many NIBRS records either could not be matched to a category in the Master Crime List, or were too vague to be matched with a single category.
- .csv downloaded from [NIBRS 2018 - Offenses as Expungeable or not](https://docs.google.com/spreadsheets/d/1htKhxWtl3Rhhd2qnqqDHiN0qfJ64iUjU0Y6XNkNG8O8/edit?usp=sharing)

In [11]:
crimelist = pd.read_csv('../data/NIBRS2018-ExpungeableOffenses.csv')
crimelist.head()

,Arrestee Offense,Expungeable,Penalty Type,Notes,Untruncated Crime Name
0,A&B on MBTA Inspector,Yes,Misdemeanor,NaN,NaN
1,A&B on MBTA Motorman,Yes,Misdemeanor,NaN,NaN
2,A&B on MBTA Operator,Yes,Misdemeanor,NaN,NaN
3,Accosting And Annoying,Yes,Misdemeanor,NaN,NaN
4,Affray,Yes,Misdemeanor,NaN,NaN


---
## Analysis on Combined Data

In [12]:
# Combine NIBRS 2018 and Master Crime List Mapping
exp = pd.merge(nibrs21, crimelist[['Arrestee Offense','Expungeable', 'Penalty Type']], on='Arrestee Offense')

In [13]:
# Get count of expungeable offenses for people 21 and younger
exp['Expungeable'].value_counts()

Yes    14130
No      1316
Name: Expungeable, dtype: int64

In [14]:
# Count of expungeable offenses for people 21 and younger
count_expungeable = exp['Expungeable'].value_counts()[0]

# Count of all incidents including people 21 or younger
count_21 = nibrs21.shape[0]

# Count of all incidents in NIBRS 2018
count_all = nibrs2018.shape[0]

In [15]:
# Get percent of expungeable offenses for people 21 and younger
count_expungeable / count_21

0.9013779025261547

In [16]:
# Get percent of expungeable offenses for people 21 and younger out of all offenses
count_expungeable / count_all

0.1270546343919721

In [17]:
exp.profile_report()

---
## Summary
- NIBRS 2018 includes **111,212** total incidents.
- Of these, **15,676** incidents involved people who were 21 or younger at the time of incident (or arrest for Boston data).
- Based on our mapping of expungement eligibility to NIBRS 2018 offenses, **90.1%** of offenses involving people 21 or younger are expungeable, which accounts for **12.7%** of ALL offenses in the NIBRS 2018 data (assuming they were the only offense).